# Review of Day 1

During Day 1, we introduced Julia basics, and saw that Julia has the following features:

- Familiar syntax
- Easy to make generic code ("write once, run everywhere")
- Julia is fast

Day 2 will be about Performance and Parallelism.
We will see how to profile code, some performance gotchas, and several ways to parallelize code using Julia.

# Simulations: profiling and performance
## Random walks

In this notebook, we will look at one of the simplest types of Monte Carlo numerical simulation, random walks.

In the simplest random walk, a particle starts at $0$ and jumps to the left ($-1$) or the right ($+1$) with equal probability.

The following is a simple implementation of a single random walk:

In [2]:
numsteps = 1000
pos = 0 
for j in 1:numsteps
            
    if rand() < 0.5
        step = -1
    else
        step = +1
    end
            
    pos += step 
end

The code seems to execute almost instantaneously, but we should **profile** (time) it:

In [4]:
@time begin
    numsteps = 1000
    pos = 0 
    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
    end
end

  0.000323 seconds (1.98 k allocations: 46.578 KB)


Although it's fast, it seems to be allocating memory unexpectedly.

Let's wrap it in a function, which is good programming practice, and allows us to have `numsteps` as a paramater.
It turns out to have an additional, important effect in Julia.

In [4]:
"""Single 1D random walk from the origin.
Returns the final position after `numsteps` steps."""
function walk(numsteps=1000)  # default value of the parameter
    
    pos = 0 
    
    for j in 1:numsteps

        if rand() < 0.5   # can replace by rand(Bool)
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
    return pos
    
end

walk

In [9]:
@time walk(1000)

  0.000017 seconds (4 allocations: 160 bytes)


2

In [7]:
@time walk(1000)

  0.000007 seconds (4 allocations: 160 bytes)


Messages: 
- Wrap everything in a function
- Run the function once, before timing only on the *second* run

Now we run it several times to collect data:

In [12]:
numsteps   = 1000
numwalkers = 10000

@time data = [walk(numsteps) for i in 1:numwalkers]  # final positions

  0.110222 seconds (12.12 k allocations: 632.654 KB)


10000-element Array{Int64,1}:
  -6
 -12
  22
  30
 -76
  24
 -18
  28
   0
  64
 -48
 -32
  34
   ⋮
  20
 -46
  26
 -12
   2
 -50
 -18
 -26
   0
   4
  36
 -66

A population of simple random walks should have mean $0$ and variance equal to the total time. Let's check it:

In [13]:
mean(data)

0.4948

In [18]:
var(data), numsteps

(1002.8467886388696,1000)

In [17]:
≈(var(data), numsteps, rtol=1e-2)

true

We can plot the histogram:

In [13]:
using Plots; gr()

histogram(data, nbins=100)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -100 
 
 
 0 
 
 
 100 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

Again, however, the high number of allocations is suspect -- in Julia, this is usually a warning that there is a "type instability". We again try wrapping it in a function, even though it seems so simple:

In [5]:
function run_walks(numwalkers, numsteps)

    data = [walk(numsteps) for i in 1:numwalkers]

    return data
end

run_walks (generic function with 1 method)

In [11]:
numsteps   = 1000
numwalkers = 10000

data = run_walks(1, 1)  # compile the function

# now profile it:
@time data = run_walks(numwalkers, numsteps);  

  0.082712 seconds (7 allocations: 78.375 KB)


In [12]:
var(data)

1004.0419771577159

# DistributedArrays 

Jump to [notebook 1a.](1a. Basics of distributed arrays.ipynb) for the basics of distributed arrays in Julia.

# Basic parallelism

In [21]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [24]:
@everywhere using DistributedArrays

In [1]:
@everywhere function walk(numsteps)
    pos = 0

    for j in 1:numsteps
        
        if rand(Bool)  # NB
            step = -1
        else
            step = +1
        end
        
        pos += step # ifelse(rand() < 0.5, -1, +1)
    end
    
    return pos
end

In [26]:
walkers = distribute(1:numwalkers);

100000-element DistributedArrays.DArray{Int64,1,UnitRange{Int64}}:
      1
      2
      3
      4
      5
      6
      7
      8
      9
     10
     11
     12
     13
      ⋮
  99989
  99990
  99991
  99992
  99993
  99994
  99995
  99996
  99997
  99998
  99999
 100000

In [27]:
walkers.indexes

2-element Array{Tuple{UnitRange{Int64}},1}:
 (1:50000,)     
 (50001:100000,)

In [55]:
@everywhere begin
    numsteps   = 10000
    numwalkers = 100000 
end

walkers = distribute(1:numwalkers);

@time positions = map( _ -> walk(numsteps), walkers)

  2.593538 seconds (11.38 k allocations: 501.654 KB)


100000-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
 -140
  170
 -142
   34
   60
  -70
   18
   20
  138
   -6
   54
  272
  -26
    ⋮
  -94
   18
  -44
  -48
  142
  -90
   10
  -54
  -22
   66
 -180
  -86

In [56]:
positions

100000-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
 -140
  170
 -142
   34
   60
  -70
   18
   20
  138
   -6
   54
  272
  -26
    ⋮
  -94
   18
  -44
  -48
  142
  -90
   10
  -54
  -22
   66
 -180
  -86

In [57]:
mean(positions)

-0.14238

In [ ]:
var(positions)

In [75]:
squared_positions = map(x->x^2, positions);

In [76]:
mean(squared_positions) ≈ var(positions)

10054.96644

# Another example: random matrices

In [1]:
using Plots; gr()

Plots.GRBackend()

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [8]:
@everywhere begin
    using DistributedArrays
    using StatsBase
    using Plots
end

In [9]:
@everywhere function stochastic(β = 2, n = 200)
    h = n ^ -(1/3)
    x = 0:h:10
    N = length(x)
    d = (-2 / h^2 .- x) + 2*sqrt(h*β) * randn(N) # diagonal
    e = ones(N - 1) / h^2                     # subdiagonal
  
    eigvals(SymTridiagonal(d, e))[N]        # smallest negative eigenvalue
end

Serial version:

In [10]:
println("Serial version")

t = 10000
p = plot()
for β = [1,2,4,10,20]
    
    z = fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end
p

Serial version


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,384.952 31.7369,384.952 32.87,384.952 34.0032,384.952 35.1364,384.952 36.2696,384.952 37.4028,384.952 38.536,384.952 39.6692,384.952 40.8023,384.952 
 41.9355,384.952 43.0687,384.952 44.2019,384.952 45.3351,384.952 46.4683,384.952 47.6014,384.952 48.7346,384.952 49.8678,384.952 51.001,384.952 52.1342,384.952 
 53.2674,384.952 54.4006,384.952 55.5337,384.952 56.6669,384.952 57.8001,384.952 58.9333,384.952 60.0665,384.952 61.1997,384.952 62.3329,384.952 63.466,384.952 
 64.5992,384.952 65.7324,384.952 66.8656,384.952 67.9988,384.952 69.132,384.952 70.2651,384.952 71.3983,384.952 72.5315,384.952 73.6647,384.952 74.7979,384.952 
 75.9311,384.952 77.0643,384.952 78.1974,384.952 79.3306,384.952 80.4638,384.952 81.597,384.952 82.7302,384.952 83.8634,384.952 84.9966,384.952 86.1297,384.952 
 87.2629,384.952 88.3961,384.952 89.5293,384.952 90.6625,384.952 91.7957,384.952 92.9288,384.952 94.062,384.952 95.1952,384.952 96.3284,384.952 97.4616,384.952 
 98.5948,384.952 99.728,384.952 100.861,384.952 101.994,384.952 103.128,384.952 104.261,384.952 105.394,384.952 106.527,384.952 107.66,384.952 108.793,384.952 
 109.927,384.952 111.06,384.952 112.193,384.952 113.326,384.952 114.459,384.952 115.593,384.952 116.726,384.952 117.859,384.952 118.992,384.952 120.125,384.952 
 121.258,384.952 122.392,384.952 123.525,384.952 124.658,384.952 125.791,384.952 126.924,384.952 128.058,384.952 129.191,384.952 130.324,384.952 131.457,384.952 
 132.59,384.952 133.724,384.952 134.857,384.952 135.99,384.952 137.123,384.952 138.256,384.952 139.389,384.952 140.523,384.952 141.656,384.952 142.789,384.952 
 143.922,383.102 145.055,384.952 146.189,384.952 147.322,384.952 148.455,384.952 149.588,384.952 150.721,384.952 151.854,383.102 152.988,384.952 154.121,384.952 
 155.254,384.952 156.387,384.952 157.52,383.102 158.654,384.952 159.787,383.102 160.92,384.952 162.053,383.102 163.186,379.403 164.32,383.102 165.453,381.253 
 166.586,381.253 167.719,381.253 168.852,379.403 169.985,383.102 171.119,384.952 172.252,375.704 173.385,383.102 174.518,375.704 175.651,383.102 176.785,366.456 
 177.918,373.854 179.051,368.306 180.184,373.854 181.317,373.854 182.45,368.306 183.584,360.907 184.717,373.854 185.85,355.358 186.983,357.208 188.116,357.208 
 189.25,344.261 190.383,351.659 191.516,340.562 192.649,336.863 193.782,335.013 194.916,316.517 196.049,305.42 197.182,312.818 198.315,322.066 199.448,307.269 
 200.581,294.322 201.715,299.871 202.848,283.225 203.981,261.03 205.114,259.18 206.247,272.127 207.381,268.428 208.514,248.082 209.647,224.038 210.78,249.932 
 211.913,225.887 213.046,211.091 214.18,188.896 215.313,199.993 216.446,194.444 217.579,177.798 218.712,142.656 219.846,153.754 220.979,138.957 222.112,127.859 
 223.245,114.912 224.378,138.957 225.511,131.558 226.645,85.3188 227.778,83.4692 228.911,59.4246 230.044,50.1767 231.177,101.965 232.311,22.4329 233.444,76.0709 
 234.577,59.4246 235.71,52.0263 236.843,61.2742 237.977,70.5221 239.11,79.7701 240.243,50.1767 241.376,72.3717 242.509,33.5304 243.642,3.93701 244.776,64.9734 
 245.909,64.9734 247.042,74.2213 248.175,107.514 249.308,85.3188 250.442,66.823 251.575,68.6725 252.708,118.611 253.841,57.575 254.974,61.2742 256.107,101.965 
 257.241,101.965 258.374,170.4 259.507,140.806 260.64,131.558 261.773,157.453 262.907,126.01 264.04,172.249 265.173,179.648 266.306,150.054 267.439,157.453 
 268.573,242.534 269.706,218.489 270.839,198.144 271.972,218.489 273.105,235.135 274.238,236.985 275.372,227.737 276.505,266.578 277.638,266.578 278.771,277.676 
 279.904,290.623 281.038,318.367 282.171,279.525 283.304,288.773 284.437,285.074 285.57,316.517 286.703,329.464 287.837,336.863 288.97,320.216 290.103,340.

A related parallel construct: `@parallel`. This does a "reduce" operation.

In [12]:
println("@parallel version")

@everywhere t = 10000

p = plot()

for β = [1,2,4,10,20]
    
    z = @parallel (+) for p = 1:nprocs()
        fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    end
    
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end

p

@parallel version


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,384.952 31.7369,384.952 32.87,384.952 34.0032,384.952 35.1364,384.952 36.2696,384.952 37.4028,384.952 38.536,384.952 39.6692,384.952 40.8023,384.952 
 41.9355,384.952 43.0687,384.952 44.2019,384.952 45.3351,384.952 46.4683,384.952 47.6014,384.952 48.7346,384.952 49.8678,384.952 51.001,384.952 52.1342,384.952 
 53.2674,384.952 54.4006,384.952 55.5337,384.952 56.6669,384.952 57.8001,384.952 58.9333,384.952 60.0665,384.952 61.1997,384.952 62.3329,384.952 63.466,384.952 
 64.5992,384.952 65.7324,384.952 66.8656,384.952 67.9988,384.952 69.132,384.952 70.2651,384.952 71.3983,384.952 72.5315,384.952 73.6647,384.952 74.7979,384.952 
 75.9311,384.952 77.0643,384.952 78.1974,384.952 79.3306,384.952 80.4638,384.952 81.597,384.952 82.7302,384.952 83.8634,384.952 84.9966,384.952 86.1297,384.952 
 87.2629,384.952 88.3961,384.952 89.5293,384.952 90.6625,384.952 91.7957,384.952 92.9288,384.952 94.062,384.952 95.1952,384.952 96.3284,384.952 97.4616,384.952 
 98.5948,384.952 99.728,384.952 100.861,384.952 101.994,384.952 103.128,384.952 104.261,384.952 105.394,384.952 106.527,384.952 107.66,384.952 108.793,384.952 
 109.927,384.952 111.06,384.952 112.193,384.952 113.326,384.952 114.459,384.952 115.593,384.952 116.726,384.952 117.859,384.952 118.992,384.952 120.125,384.952 
 121.258,384.952 122.392,384.952 123.525,384.952 124.658,384.952 125.791,384.952 126.924,384.952 128.058,384.952 129.191,384.952 130.324,384.952 131.457,384.952 
 132.59,384.952 133.724,384.952 134.857,384.952 135.99,384.952 137.123,384.952 138.256,384.952 139.389,384.952 140.523,384.952 141.656,384.952 142.789,384.952 
 143.922,384.952 145.055,384.139 146.189,384.952 147.322,384.952 148.455,384.952 149.588,384.952 150.721,384.546 151.854,384.546 152.988,383.733 154.121,383.733 
 155.254,384.546 156.387,384.139 157.52,384.546 158.654,383.733 159.787,384.139 160.92,382.921 162.053,384.139 163.186,382.108 164.32,382.108 165.453,382.515 
 166.586,381.296 167.719,380.484 168.852,382.108 169.985,377.64 171.119,378.859 172.252,377.234 173.385,377.64 174.518,371.547 175.651,377.234 176.785,370.735 
 177.918,372.766 179.051,367.892 180.184,366.267 181.317,368.704 182.45,363.017 183.584,353.268 184.717,358.549 185.85,358.955 186.983,354.081 188.116,352.862 
 189.25,343.52 190.383,339.458 191.516,338.239 192.649,338.645 193.782,324.022 194.916,323.616 196.049,310.211 197.182,310.617 198.315,311.43 199.448,304.118 
 200.581,289.495 201.715,279.34 202.848,267.56 203.981,275.278 205.114,250.094 206.247,239.126 207.381,244.407 208.514,228.159 209.647,227.347 210.78,209.068 
 211.913,218.816 213.046,189.976 214.18,183.883 215.313,173.322 216.446,174.947 217.579,168.854 218.712,150.169 219.846,116.048 220.979,104.674 222.112,109.143 
 223.245,112.392 224.378,90.0512 225.511,75.4281 226.645,65.6793 227.778,63.6483 228.911,64.0545 230.044,65.2731 231.177,56.7429 232.311,45.3693 233.444,28.309 
 234.577,5.5618 235.71,58.3677 236.843,19.7788 237.977,28.309 239.11,14.4982 240.243,3.93701 241.376,11.2486 242.509,17.7478 243.642,16.9354 244.776,30.7462 
 245.909,31.9648 247.042,24.6532 248.175,14.9044 249.308,14.4982 250.442,36.8391 251.575,40.9011 252.708,56.3367 253.841,69.3351 254.974,60.3987 256.107,73.3971 
 257.241,98.9876 258.374,112.392 259.507,104.674 260.64,113.611 261.773,124.984 262.907,126.609 264.04,148.138 265.173,130.671 266.306,162.355 267.439,160.324 
 268.573,170.479 269.706,192.413 270.839,215.567 271.972,210.286 273.105,220.035 274.238,224.909 275.372,242.376 276.505,252.531 277.638,258.218 278.771,265.123 
 279.904,265.529 281.038,287.464 282.171,285.027 283.304,295.994 284.437,312.648 285.57,310.211 286.703,308.586 287.837,311.024 288.97,317.117 290.103,324.428 
 291.236,3